In [1]:
import numpy as np
import pandas as pd
import QUANTAXIS as qa

QUANTAXIS>> start QUANTAXIS
QUANTAXIS>> failed to import pyecharts_snapshot
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/lml/utils.py", line 43, in do_import
    plugin_module = __import__(plugin_module_name)
ModuleNotFoundError: No module named 'pyecharts_snapshot'
QUANTAXIS>> Expand macros in /usr/local/lib/python3.6/site-packages/macropy/core/hquotes.py
QUANTAXIS>> Finding macros in 'macropy.core.hquotes'
QUANTAXIS>> Importing macros from 'macropy.core.quotes' into 'macropy.core.hquotes'
QUANTAXIS>> Expand macros in /usr/local/lib/python3.6/site-packages/macropy/core/quotes.py
QUANTAXIS>> Finding macros in 'macropy.core.quotes'
QUANTAXIS>> Expand macros in /usr/local/lib/python3.6/site-packages/macropy/core/failure.py
QUANTAXIS>> Finding macros in 'macropy.core.failure'
QUANTAXIS>> Importing macros from 'macropy.core.hquotes' into 'macropy.core.failure'
QUANTAXIS>> Expand macros in /usr/local/lib/python3.6/site-packages/cffi/api.py
QUANTAXIS>> Fi

In [2]:
can = qa.QA_fetch_stock_day_adv('000002', start='2017-06-01', end='2018-05-29').to_qfq()

In [3]:
raw = can.data.loc[:, ['open', 'close', 'high', 'low']].values

In [4]:
dates = can.data.date

In [5]:
from pyecharts import Grid, Bar, Line, Kline, Overlap

kline = Kline()
kline.add("日K", dates.tolist(), raw, is_datazoom_show=True)

overlap = Overlap()
overlap.add(kline)
overlap

In [16]:
mrg = []
mrg_cnt = []
mrg_date = []
raw = raw
raw_date = dates.tolist()
raw_idx = 0
mrg_idx = 0

while True:
    if raw_idx >= len(raw):
        break
    if raw_idx == 0:
        mrg.append(raw[raw_idx])
        mrg_date.append(raw_date[raw_idx])
        mrg_cnt.append(1)
        raw_idx += 1
        continue
    if (mrg[mrg_idx][2] >= raw[raw_idx][2] and mrg[mrg_idx][3] <= raw[raw_idx][3]) or (mrg[mrg_idx][2] <= raw[raw_idx][2] and mrg[mrg_idx][3] >= raw[raw_idx][3]):
        if mrg_idx == 0 or mrg[mrg_idx-1][2] <= mrg[mrg_idx][2]:
            mrg[mrg_idx][0] = max(mrg[mrg_idx][0], raw[raw_idx][0])
            mrg[mrg_idx][1] = max(mrg[mrg_idx][1], raw[raw_idx][1])
            mrg[mrg_idx][2] = max(mrg[mrg_idx][2], raw[raw_idx][2])
            mrg[mrg_idx][3] = max(mrg[mrg_idx][3], raw[raw_idx][3])
        else:
            mrg[mrg_idx][0] = min(mrg[mrg_idx][0], raw[raw_idx][0])
            mrg[mrg_idx][1] = min(mrg[mrg_idx][1], raw[raw_idx][1])
            mrg[mrg_idx][2] = min(mrg[mrg_idx][2], raw[raw_idx][2])
            mrg[mrg_idx][3] = min(mrg[mrg_idx][3], raw[raw_idx][3])
        mrg_date[mrg_idx] = raw_date[raw_idx]
        mrg_cnt[mrg_idx] += 1
        raw_idx += 1
        continue
    mrg.append(raw[raw_idx])
    mrg_date.append(raw_date[raw_idx])
    mrg_cnt.append(1)
    raw_idx += 1
    mrg_idx += 1

[(mrg_date[idx], mrg[idx][2], mrg[idx][3], mrg_cnt[idx]) for idx in range(len(mrg))]

[(Timestamp('2017-06-01 00:00:00'), 20.666863798256152, 20.241341099555513, 1),
 (Timestamp('2017-06-02 00:00:00'), 20.59916700528105, 20.096276543180295, 1),
 (Timestamp('2017-06-05 00:00:00'), 20.560483123580994, 20.028579750205193, 1),
 (Timestamp('2017-06-06 00:00:00'), 20.744231561656267, 20.144631395305364, 1),
 (Timestamp('2017-06-13 00:00:00'), 22.04981256903324, 20.792586413781343, 2),
 (Timestamp('2017-06-14 00:00:00'), 21.179425230781924, 20.61850894613108, 1),
 (Timestamp('2017-06-15 00:00:00'), 20.376734685505717, 20.13496042488035, 1),
 (Timestamp('2017-06-21 00:00:00'), 20.72488962080624, 20.212328188280466, 4),
 (Timestamp('2017-06-22 00:00:00'), 21.56626404778251, 20.40574759678076, 1),
 (Timestamp('2017-06-23 00:00:00'), 23.27802581301009, 21.082715526531782, 1),
 (Timestamp('2017-06-27 00:00:00'), 25.608729685438604, 24.341832559761695, 2),
 (Timestamp('2017-06-28 00:00:00'), 25.038142430362743, 23.964664713186124, 1),
 (Timestamp('2017-06-30 00:00:00'), 24.554593909

In [20]:
kline = Kline()
kline.add("日K", mrg_date, mrg, is_datazoom_show=True, 
          mark_point=[{'coord':[mrg_date[-10], mrg[-10][2] * 1.03], 'name':'test'}], 
          mark_point_symbol='circle', mark_point_symbolsize=10)
kline
#overlap = Overlap()
#overlap.add(kline)
#overlap

In [25]:
def higher(x, y):
    return x[2] >= y[2] and x[3] >= y[3]

def lower(x, y):
    return x[2] <= y[2] and x[3] <= y[3]

end = []
last = 0
for idx, cell in enumerate(mrg):
    insert = False
    if idx < 2:
        continue
    if (higher(mrg[idx-1], mrg[idx-2]) and higher(mrg[idx-1], cell)):
        end.append([idx-1, 1, True])
        insert = True
    elif (lower(mrg[idx-1], mrg[idx-2]) and lower(mrg[idx-1], cell)):
        end.append([idx-1, 0, True])
        insert = True
    
    if insert and len(end)>1 and end[-1][1] == end[-2][1]:
        if ((end[-1][1] == 1 and mrg[end[-1][0]][2] > mrg[end[-2][0]][2]) or 
            (end[-1][1] == 0 and mrg[end[-1][0]][3] < mrg[end[-2][0]][3])):
            if not(idx-2 == last and mrg_cnt[last] + mrg_cnt[last+1] > 2 or idx-3 > last) and len(end) > 2:
                end.pop(-2)
            else:
                end[-2][2] = False
        else:
            end.pop()
            insert = False
    elif insert and not(idx-2 == last and mrg_cnt[last] + mrg_cnt[last+1] > 2 or idx-3 > last):
        end.pop()
        insert = False
    
    if insert:
        last = idx

[(str(mrg_date[item[0]]), 'top' if item[1] else 'bottom', 'valid' if item[2] else 'not') for item in end]

[('2017-06-13 00:00:00', 'top', 'not'),
 ('2017-06-27 00:00:00', 'top', 'valid'),
 ('2017-07-05 00:00:00', 'bottom', 'valid'),
 ('2017-07-12 00:00:00', 'top', 'valid'),
 ('2017-07-27 00:00:00', 'bottom', 'not'),
 ('2017-08-08 00:00:00', 'bottom', 'not'),
 ('2017-08-17 00:00:00', 'bottom', 'valid'),
 ('2017-08-28 00:00:00', 'top', 'valid'),
 ('2017-09-04 00:00:00', 'bottom', 'valid'),
 ('2017-09-11 00:00:00', 'top', 'not'),
 ('2017-09-15 00:00:00', 'top', 'valid'),
 ('2017-09-25 00:00:00', 'bottom', 'not'),
 ('2017-09-29 00:00:00', 'bottom', 'valid'),
 ('2017-10-13 00:00:00', 'top', 'not'),
 ('2017-11-01 00:00:00', 'top', 'valid'),
 ('2017-11-07 00:00:00', 'bottom', 'not'),
 ('2017-11-14 00:00:00', 'bottom', 'valid'),
 ('2017-11-22 00:00:00', 'top', 'valid'),
 ('2017-12-04 00:00:00', 'bottom', 'not'),
 ('2017-12-08 00:00:00', 'bottom', 'valid'),
 ('2017-12-14 00:00:00', 'top', 'not'),
 ('2017-12-25 00:00:00', 'top', 'not'),
 ('2018-01-08 00:00:00', 'top', 'not'),
 ('2018-01-24 00:00:00'

In [26]:
valid = [x for x in end if x[2]]
valid

[[10, 1, True],
 [15, 0, True],
 [20, 1, True],
 [41, 0, True],
 [46, 1, True],
 [51, 0, True],
 [59, 1, True],
 [69, 0, True],
 [82, 1, True],
 [88, 0, True],
 [94, 1, True],
 [103, 0, True],
 [134, 1, True],
 [144, 0, True],
 [148, 1, True],
 [163, 0, True],
 [170, 1, True],
 [172, 0, True],
 [176, 1, True],
 [188, 0, True]]